In [ ]:
# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [1]:
import json
import os
import pandas as pd
import pyarrow

import sys
from os import path
import numpy

from dotenv import load_dotenv
from sqlalchemy import create_engine


load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [2]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [6]:
import prepare_data

In [163]:
COMPANY_NAME = 'PL'
COMPANY_IDENTIFIER = 'PL'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

INVENTORY_DATE = '10/17/2021'

In [164]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pd.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pd.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)

In [165]:
deduped_sales_receipts_dataframe = prepare_data.dedupe_sales_transactions(company_sales_transactions_dataframe)

In [166]:
df_in = company_incoming_transfer_packages_dataframe
df_out = company_outgoing_transfer_packages_dataframe
df_sales = deduped_sales_receipts_dataframe
df_inventory = company_inventory_packages_dataframe

In [167]:
pd.options.display.float_format = '{:,.2f}'.format

## Sales

In [168]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988992 entries, 0 to 988991
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   id                        988992 non-null  object             
 1   license_number            988992 non-null  object             
 2   receipt_number            988992 non-null  object             
 3   rt_type                   988992 non-null  object             
 4   sales_customer_type       988992 non-null  object             
 5   sales_datetime            988992 non-null  datetime64[ns, UTC]
 6   total_packages            988992 non-null  int64              
 7   rt_total_price            988992 non-null  float64            
 8   tx_type                   988992 non-null  object             
 9   tx_package_id             988992 non-null  object             
 10  tx_package_label          988992 non-null  object             
 11  

In [13]:
# df_sales[df_sales['tx_quantity_sold'] <1]

In [169]:
df_sales['per_unit'] = df_sales['tx_total_price'] / df_sales['tx_quantity_sold']
df_sales['year_month'] = df_sales['sales_datetime'].dt.strftime("%Y-%m")

In [172]:
s_total_count = df_sales.groupby('year_month')['tx_total_price'].count()
df_total_count = pd.Series(s_total_count).to_frame()
df_total_count = df_total_count.reset_index()
df_total_count.rename(columns={'tx_total_price':'total_count'}, inplace=True)

In [175]:
s_revenue = df_sales.groupby('year_month')['tx_total_price'].sum()
df_revenue = pd.Series(s_revenue).to_frame()
df_revenue = df_revenue.reset_index()
df_revenue.rename(columns={'tx_total_price': 'revenue'}, inplace=True)

## Incoming

In [179]:
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18784 entries, 0 to 18783
Data columns (total 28 columns):
 #   Column                                 Non-Null Count  Dtype              
---  ------                                 --------------  -----              
 0   package_row_id                         18784 non-null  object             
 1   delivery_type                          18784 non-null  object             
 2   license_number                         18784 non-null  object             
 3   manifest_number                        18784 non-null  object             
 4   created_date                           18784 non-null  object             
 5   received_datetime                      18715 non-null  datetime64[ns, UTC]
 6   shipper_facility_license_number        18784 non-null  object             
 7   shipper_facility_name                  18784 non-null  object             
 8   recipient_facility_license_number      18784 non-null  object             
 9   recipi

In [180]:
df_in['per_unit_incoming'] = df_in['shipper_wholesale_price'] / df_in['shipped_quantity']

In [181]:
average_incoming_package_id = df_in.groupby('package_id')['per_unit_incoming'].mean()

In [183]:
df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
df_avg_incoming_price = df_avg_incoming_price.reset_index()

In [211]:
average_incoming_product = df_in.groupby('product_name')['per_unit_incoming'].mean()

In [212]:
df_avg_product = pd.Series(average_incoming_product).to_frame()

In [213]:
df_avg_product = df_avg_product.reset_index()

In [215]:
df_avg_product.rename(columns={'per_unit_incoming':'per_unit_product'}, inplace=True)

In [218]:
df_avg_product.head(2)

,product_name,per_unit_product
0,"""1943"" - Balanced Pre-Packaged 1/8th",0.00
1,(AL/FF) - (Melonade) - .5g Cured Resin Cartridge,17.00


## COGS

In [225]:
df_cogs = pd.merge(df_sales, df_avg_incoming_price, left_on='tx_package_id', right_on='package_id', how='left')

In [226]:
df_cogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988992 entries, 0 to 988991
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   id                        988992 non-null  object             
 1   license_number            988992 non-null  object             
 2   receipt_number            988992 non-null  object             
 3   rt_type                   988992 non-null  object             
 4   sales_customer_type       988992 non-null  object             
 5   sales_datetime            988992 non-null  datetime64[ns, UTC]
 6   total_packages            988992 non-null  int64              
 7   rt_total_price            988992 non-null  float64            
 8   tx_type                   988992 non-null  object             
 9   tx_package_id             988992 non-null  object             
 10  tx_package_label          988992 non-null  object             
 11  

In [228]:
df_cogs_null = df_cogs[df_cogs['per_unit_incoming'].isnull()]

In [229]:
df_cogs_null_product = pd.merge(df_cogs_null, df_avg_product, left_on='tx_product_name', right_on='product_name', how='left')

In [232]:
df_cogs_product = df_cogs_null_product[df_cogs_null_product['per_unit_product'].notnull()]

In [234]:
df_cogs_product['total_product'] = df_cogs_product['tx_quantity_sold'] * df_cogs_product['per_unit_product']

C:\Users\Rachel\GitHub\venvs\bespoke\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [236]:
df_cogs_product.groupby('year_month')['total_product'].count()

year_month
2020-01    7137
2020-02     668
2020-03      64
2020-04      47
2020-05       4
2020-06       5
2020-07     186
2020-08     280
2020-09     837
2020-10    1991
2020-11    1685
2020-12    1633
2021-01     442
2021-02     435
2021-03     104
2021-04     374
2021-05     697
2021-06     526
2021-07     892
2021-08     169
2021-10     116
Name: total_product, dtype: int64

In [239]:
52882 + 892

53774

In [241]:
53774/63441

0.8476222001544742

In [186]:
df_cogs['total_incoming'] = df_cogs['per_unit_incoming'] * df_cogs['tx_quantity_sold']

In [221]:
df_cogs = pd.merge(df_cogs, df_avg_product, left_on='tx_product_name', right_on='product_name', how='left')

In [224]:
# df_cogs[df_cogs['per_unit_incoming'].isnull() & df_cogs['per_unit_product'].notnull()]

In [187]:
s_cogs_count = df_cogs.groupby('year_month')['total_incoming'].count()
df_cogs_count = pd.Series(s_cogs_count).to_frame()
df_cogs_count = df_cogs_count.reset_index()
df_cogs_count.rename(columns={'total_incoming':'count_incoming'}, inplace=True)

In [190]:
s_cogs = df_cogs.groupby('year_month')['total_incoming'].sum()
df_cogs_2 = pd.Series(s_cogs).to_frame()
df_cogs_2 = df_cogs_2.reset_index()

## Summary

In [193]:
df_summary = pd.merge(df_revenue, df_cogs_2)

In [194]:
df_summary['margin_$'] = df_summary['revenue'] - df_summary['total_incoming']
df_summary['margin_%'] = df_summary['margin_$'] / df_summary['revenue']

In [197]:
df_summary = pd.merge(df_summary, df_cogs_count)
df_summary = pd.merge(df_summary, df_total_count)

In [199]:
df_summary['coverage'] = df_summary['count_incoming'] / df_summary['total_count']

In [200]:
df_summary

,year_month,revenue,total_incoming,margin_$,margin_%,count_incoming,total_count,coverage
0,2020-01,"1,404,088.07","323,586.13","1,080,501.94",0.77,20467,43877,0.47
1,2020-02,"1,342,798.27","537,508.72","805,289.55",0.60,31606,43366,0.73
2,2020-03,"1,575,122.34","694,260.26","880,862.08",0.56,42528,49114,0.87
3,2020-04,"1,797,969.95","777,819.93","1,020,150.02",0.57,48656,54122,0.90
4,2020-05,"2,045,620.79","849,484.47","1,196,136.32",0.58,55630,59021,0.94
5,2020-06,"2,017,956.21","849,524.95","1,168,431.26",0.58,53637,56866,0.94
6,2020-07,"2,076,954.20","782,518.99","1,294,435.21",0.62,52251,59013,0.89
7,2020-08,"1,976,309.93","744,093.45","1,232,216.48",0.62,48038,53862,0.89
8,2020-09,"1,476,503.57","553,883.88","922,619.69",0.62,35327,40628,0.87
9,2020-10,"1,793,659.60","655,092.02","1,138,567.58",0.63,43224,51365,0.84


## Inventory

In [201]:
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5090 entries, 0 to 5089
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   license_number         5090 non-null   object 
 1   package_id             5090 non-null   object 
 2   package_label          5090 non-null   object 
 3   type                   5090 non-null   object 
 4   packaged_date          5090 non-null   object 
 5   package_type           5090 non-null   object 
 6   product_name           5090 non-null   object 
 7   product_category_name  5090 non-null   object 
 8   quantity               5090 non-null   float64
 9   unit_of_measure        5090 non-null   object 
 10  is_testing_sample      5090 non-null   bool   
 11  is_trade_sample        5090 non-null   bool   
 12  is_on_hold             5090 non-null   bool   
 13  archived_date          0 non-null      object 
 14  finished_date          6 non-null      object 
dtypes: b

In [242]:
df_inventory_incoming = pd.merge(df_inventory, df_avg_incoming_price, on='package_id', how='left')

In [243]:
df_inventory_incoming['total_price'] = df_inventory_incoming['quantity'] * df_inventory_incoming['per_unit_incoming']
inventory_value = df_inventory_incoming['total_price'].sum()
print(f'{inventory_value:,.2f}')

2,091,299.11


In [244]:
inv_count_incoming = df_inventory_incoming['per_unit_incoming'].count()
inv_count_total = df_inventory_incoming['quantity'].count()
inventory_coverage = inv_count_incoming / inv_count_total
print(f'{inventory_coverage:,.0%}')

86%


86%


# Notes

In [ ]:
# df_in[df_in.duplicated('package_id')]
# df_sales[df_sales.duplicated('tx_package_id')]

In [48]:
# pd.merge(df_sales, df_in[['package_id','per_unit_incoming']], left_on='tx_package_id', right_on='package_id', how='left')

In [49]:
# df_sales.merge(df_in['package_id','per_unit_incoming'], left_on='tx_package_id', right_on='package_id', how='left')

In [58]:
# incoming_transfer_simp = company_incoming_transfer_packages_dataframe[[
# 'license_number',
# 'created_date',
# 'package_id',
# 'product_category_name',
# 'product_name',
# 'shipper_wholesale_price',
# 'shipped_quantity',
# 'shipped_unit_of_measure',
# 'item_unit_weight',
# 'item_unit_weight_unit_of_measure_name']]

In [30]:
# incoming_transfer_simp

In [59]:
# outgoing_transfer_simp = company_outgoing_transfer_packages_dataframe[[
# 'license_number',
# 'created_date',
# 'package_id',
# 'product_category_name',
# 'product_name',
# 'shipper_wholesale_price',
# 'shipped_quantity',
# 'shipped_unit_of_measure',
# 'item_unit_weight',
# 'item_unit_weight_unit_of_measure_name']]

In [53]:
# company_sales_transactions_dataframe.to_csv('hpcc_all_sales_10_13_21.csv')

In [110]:
# company_sales_transactions_dataframe.info()

In [60]:
# sales_simp = company_sales_transactions_dataframe[[
# 'license_number',
# 'sales_datetime',
# 'tx_package_id',
# 'tx_product_name',
# 'tx_product_category_name',
# 'tx_unit_of_measure',
# 'tx_quantity_sold',
# 'tx_total_price']]

In [61]:
# inventory_simp = company_inventory_packages_dataframe[[
# 'license_number',
# 'package_id',
# 'packaged_date',
# 'product_name',
# 'product_category_name',
# 'quantity',
# 'unit_of_measure'
# ]]

In [109]:
# sales_simp['year_month'] = sales_simp['sales_datetime'].dt.strftime("%Y-%m")

In [108]:
# sales_simp.groupby(by=['year_month'], as_index=False).count()

In [51]:
# sales_drop = sales_simp.drop(['sales_datetime', 'month_year'], axis=1)

In [52]:
# sales_simp.to_csv('./hpcc_sales_2021_10_13.csv')

In [ ]:
# INVENTORY_DATES = [
#     '09/30/2020',
#     '10/31/2020',
#     '11/30/2020',
#     '12/31/2020',
#     '01/31/2021',
#     '02/28/2021',
#     '03/31/2021',
#     '04/30/2021',
#     '05/31/2021',
#     '06/30/2021',
#     '07/31/2021',
#     '08/31/2021',
#     '09/30/2021',
# ]

In [12]:
# %autoreload 2

# sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
# sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

# from util import active_inventory_util as util

In [13]:
# d = util.Download()
# d.download_dataframes(
#     incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
#     outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
#     sales_transactions_dataframe=company_sales_transactions_dataframe,
# )

In [19]:
# q = util.Query()
# q.inventory_dates = INVENTORY_DATES
# q.company_name = COMPANY_NAME

# id_to_history = util.get_histories(d)
# util.print_counts(id_to_history)
# util.create_inventory_xlsx(id_to_history, q)

In [20]:
# computed_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, INVENTORY_DATE)
# computed_inventory_packages_dataframe = pandas.DataFrame(
#     computed_inventory_package_records,
#     columns=[
#         'package_id',
#         'Arrived Date',
#         'Product Category',
#         'Product Name',
#         'Current Quantity',
#         'Sold Date',
#     ]
# )
# computed_inventory_packages_dataframe

In [21]:
# date_to_inventory_dataframe = {}
# for date, inventory_records in date_to_inventory_records.items():
#     date_to_inventory_dataframe[date] = pandas.DataFrame(
#         inventory_records,
#         columns=[
#             'package_id',
#             'Arrived Date',
#             'Product Category',
#             'Product Name',
#             'Current Quantity',
#             'Sold Date',
#         ]
#     )
    
# date_to_inventory_dataframe[list(date_to_inventory_dataframe.keys())[0]]

In [17]:
# raw_incoming_transfer_packages_dataframe = pandas.read_excel('data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx', header=0)
# len(raw_incoming_transfer_packages_dataframe.index), raw_incoming_transfer_packages_dataframe.columns

In [88]:
# incoming_transfer_packages_dataframe = raw_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
#     'delivery_type',
#     'manifest_number',
#     'created_date',
#     'received_datetime',
#     'shipper_facility_license_number',
#     'shipper_facility_name',
#     'recipient_facility_license_number',
#     'recipient_facility_name',
#     'shipment_type_name',
#     'shipment_transaction_type',
#     'package_id',
#     'package_label',
#     'type',
#     'product_category_name',
#     'product_name',
#     'shipper_wholesale_price',
#     'shipped_quantity',
#     'package_lab_results_status',
#     'shipment_package_state',
#     'is_testing_sample',
#     'is_trade_sample'
# ]]

In [18]:
# for date, inventory_dataframe in date_to_inventory_dataframe.items():
#     print(date)
#     print(f'# of packages in inventory: {len(inventory_dataframe.index)}')

#     inventory_with_incoming_transfer_packages_dataframe = inventory_dataframe.astype({'package_id': 'int64'}).merge(incoming_transfer_packages_dataframe, on='package_id', how='inner', suffixes=('_l', '_r'))
# #     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')
    
#     inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

#     total_valuation_cost = 0
#     import math
#     for inventory_with_cost_record in inventory_with_cost_records:
#         incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
#         if math.isnan(incoming_shipped_price):
#             incoming_shipped_price = 0
#         incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
#         current_quantity = inventory_with_cost_record['Current Quantity']
#         total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

#     print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
#     print('')

In [ ]:
# import json
# import pandas
# import numpy
# import os
# import sys
# from os import path